# FINRA API Authentication Debug

This notebook tests FINRA API authentication to diagnose the 400 Bad Request error.

In [12]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables from root .env
load_dotenv('.env')

FINRA_API_KEY = os.getenv('FINRA_API_KEY')
FINRA_API_SECRET = os.getenv('FINRA_API_SECRET')

print(f"API Key loaded: {FINRA_API_KEY[:10]}..." if FINRA_API_KEY else "API Key NOT FOUND")
print(f"API Secret loaded: {FINRA_API_SECRET[:5]}..." if FINRA_API_SECRET else "API Secret NOT FOUND")

API Key loaded: 3dd66476e1...
API Secret loaded: e9ZIg...


## Test 1: Token URL with grant_type in body only

FINRA OAuth2 typically expects `grant_type` in the request body, not the URL.

In [13]:
# Token URL WITHOUT grant_type in query string
TOKEN_URL = "https://ews.fip.finra.org/fip/rest/ews/oauth2/access_token"

response = requests.post(
    TOKEN_URL,
    auth=(FINRA_API_KEY, FINRA_API_SECRET),
    data={"grant_type": "client_credentials"},
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Headers: {dict(response.headers)}")
print(f"Response: {response.text[:500] if response.text else 'No response body'}")

Status: 200
Headers: {'Date': 'Sun, 21 Dec 2025 23:26:13 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Server': 'cloudflare', 'Set-Cookie': 'AWSALB=kWsF/WtV+/XKjTobcUCwLweRA0DXFPaqiiQ9vjQjl2F2DcyigvzC+0Bvkw7LUK/e9gB/hE9Dk5TQAf+4AD1mCcX6fjFfbGi4zIv8tEesYrrWzNERJo5SS0d9Soqg; Expires=Sun, 28 Dec 2025 23:26:13 GMT; Path=/, AWSALBCORS=kWsF/WtV+/XKjTobcUCwLweRA0DXFPaqiiQ9vjQjl2F2DcyigvzC+0Bvkw7LUK/e9gB/hE9Dk5TQAf+4AD1mCcX6fjFfbGi4zIv8tEesYrrWzNERJo5SS0d9Soqg; Expires=Sun, 28 Dec 2025 23:26:13 GMT; Path=/; SameSite=None; Secure, AppSession=7072B19E-DEC4-11F0-9848-236B4F8A3EFD;Domain=.finra.org;Path=/;Secure;HttpOnly;Max-Age=315360000, AppSession=7072B19E-DEC4-11F0-9848-236B4F8A3EFD;Domain=.finra.org;Path=/;Secure;HttpOnly;Max-Age=315360000, __cf_bm=PpecnEsx5EuxZyav.B2wfkNHAhyLw33qPIHReNOr5fo-1766359572.949866-1.0.1.1-LwxeCJ7kUB6XpcvOGN4pdxbopEPNOUo8honmOd6NnYPj5DDHofj2zKaHTNHO0ce.yqGTQd1FBVyMUhHKH9xECzQh.87xciJNlMxXgDqN9KZOkEEpT5OJDYGtB

## Test 2: Token URL with grant_type in query string

Some APIs expect it in the URL instead.

In [14]:
# Token URL WITH grant_type in query string
TOKEN_URL_WITH_GRANT = "https://ews.fip.finra.org/fip/rest/ews/oauth2/access_token?grant_type=client_credentials"

response = requests.post(
    TOKEN_URL_WITH_GRANT,
    auth=(FINRA_API_KEY, FINRA_API_SECRET),
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Response: {response.text[:500] if response.text else 'No response body'}")

Status: 200
Response: {"access_token":"*AAJTSQACMDIABHR5cGUAA0pXVAACUzEAAjAx*eyJ0eXAiOiJKV1QiLCJjdHkiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.ZXlKMGVYQWlPaUpLVjFRaUxDSmxibU1pT2lKQk1USTRRMEpETFVoVE1qVTJJaXdpWVd4bklqb2laR2x5SW4wLi5MQmN5MjBfejg1c0ZnZXhvTE9nT0RnLkZrU2U0elhhZ1Vod2tKT1d2U1p0M3lsVEV0LWpPeDlvaTJyMXRRX3ZRSlVRN3RkRDZ3N3pQdHlhNGM1Tm1pbjNWdFpmOW9wUUJiLVdIWU8tS0ZDR05Pdk8zc2lTdUpia3RLbzNMM3lJZ042TWVfSFBwdGh6Ql9WVEFaOW9rb0l2V0NSdzB2V1FFeWNCNm1CMi1PUEdrSXN6S1FEZklRdXdBU3JOeFRDelI2NzJ1UDlWSmlYWU9ZWHphQi01RWZzWHpNbi1tV2xQeDBNQTJk


## Test 3: Using headers instead of Basic Auth

Some FINRA endpoints use custom headers for authentication.

In [15]:
TOKEN_URL = "https://ews.fip.finra.org/fip/rest/ews/oauth2/access_token"

# Try with X-API-KEY headers instead of Basic Auth
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "X-API-KEY": FINRA_API_KEY,
    "X-API-SECRET": FINRA_API_SECRET
}

response = requests.post(
    TOKEN_URL,
    headers=headers,
    data={"grant_type": "client_credentials"},
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Response: {response.text[:500] if response.text else 'No response body'}")

Status: 400
Response: {"error_message":"Client credentials is required","error":"invalid_client"}


## Test 4: Direct API call without token (API Key auth)

Some FINRA APIs allow direct API key authentication without OAuth token.

In [16]:
# Try direct API call with API key headers
OTC_URL = "https://api.finra.org/data/group/otcMarket/name/weeklySummary"

headers = {
    "Accept": "application/json",
    "X-API-KEY": FINRA_API_KEY
}

response = requests.get(
    OTC_URL,
    headers=headers,
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Response: {response.text[:500] if response.text else 'No response body'}")

Status: 200
Response: [{"totalWeeklyShareQuantity":6489206414,"issueSymbolIdentifier":null,"issueName":null,"lastUpdateDate":"2023-12-11","lastReportedDate":"2023-11-10","tierDescription":"Not Applicable","initialPublishedDate":"2023-12-11","tierIdentifier":"NMS","summaryStartDate":"2023-11-06","totalNotionalSum":347503970963,"totalWeeklyTradeCount":66391371,"weekStartDate":"2023-11-06","MPID":null,"firmCRDNumber":null,"productTypeCode":null,"marketParticipantName":null,"summaryTypeCode":"ATS_W_VOL_STATS"},{"totalWee


## Test 5: Check FINRA Gateway API directly

Try the FINRA Gateway API endpoint format.

In [17]:
# FINRA Gateway API format
GATEWAY_URL = "https://api.finra.org/data/group/otcMarket/name/weeklySummary"

# Using Basic Auth with the API credentials
response = requests.post(
    GATEWAY_URL,
    auth=(FINRA_API_KEY, FINRA_API_SECRET),
    headers={"Accept": "application/json", "Content-Type": "application/json"},
    json={},  # Empty body for now
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Response: {response.text[:1000] if response.text else 'No response body'}")

Status: 200
Response: [{"totalWeeklyShareQuantity":6489206414,"issueSymbolIdentifier":null,"issueName":null,"lastUpdateDate":"2023-12-11","lastReportedDate":"2023-11-10","tierDescription":"Not Applicable","initialPublishedDate":"2023-12-11","tierIdentifier":"NMS","summaryStartDate":"2023-11-06","totalNotionalSum":347503970963,"totalWeeklyTradeCount":66391371,"weekStartDate":"2023-11-06","MPID":null,"firmCRDNumber":null,"productTypeCode":null,"marketParticipantName":null,"summaryTypeCode":"ATS_W_VOL_STATS"},{"totalWeeklyShareQuantity":21779,"issueSymbolIdentifier":"BRKR","issueName":"Bruker Corporation Common Stock","lastUpdateDate":"2023-11-27","lastReportedDate":"2023-11-10","tierDescription":"NMS Tier 1","initialPublishedDate":"2023-11-27","tierIdentifier":"NMS","summaryStartDate":"2023-11-06","totalNotionalSum":1257506,"totalWeeklyTradeCount":241,"weekStartDate":"2023-11-06","MPID":"JPBX","firmCRDNumber":null,"productTypeCode":null,"marketParticipantName":"JPBX JPB-X","summaryTypeCo

## Test 6: Correct FINRA CAT/OTC API format

Based on FINRA documentation, try the proper request format.

In [18]:
import base64

# Encode credentials for Authorization header
credentials = f"{FINRA_API_KEY}:{FINRA_API_SECRET}"
encoded_credentials = base64.b64encode(credentials.encode()).decode()

TOKEN_URL = "https://ews.fip.finra.org/fip/rest/ews/oauth2/access_token"

headers = {
    "Authorization": f"Basic {encoded_credentials}",
    "Content-Type": "application/x-www-form-urlencoded"
}

response = requests.post(
    TOKEN_URL,
    headers=headers,
    data="grant_type=client_credentials",
    timeout=30
)

print(f"Status: {response.status_code}")
print(f"Response: {response.text[:500] if response.text else 'No response body'}")

if response.status_code == 200:
    token_data = response.json()
    print(f"\nAccess Token: {token_data.get('access_token', 'N/A')[:50]}...")

Status: 200
Response: {"access_token":"*AAJTSQACMDIABHR5cGUAA0pXVAACUzEAAjAx*eyJ0eXAiOiJKV1QiLCJjdHkiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.ZXlKMGVYQWlPaUpLVjFRaUxDSmxibU1pT2lKQk1USTRRMEpETFVoVE1qVTJJaXdpWVd4bklqb2laR2x5SW4wLi5qeFJzU0hhRzZXVWhOZnZ2VWhKSDBRLmRnUTE4TnMyaW90UFltZDR3c1dnVFJwOUlKOVRiTFBZc1l4eDg1NkRTcTJOSU9GZUZmNzdQU01UZjljMlZWVXlYVXU4SXdoVGpaeVBYcUw0RUdYUFpoS3N3RllPQlU3aUtxdkxEX1FMbjJLR3RKUFVnV1JlNDhDeDJnb0JacDJpZ19yeFBLM3dzX2lwS0xvNUViOTNITXZzalkwbExRTHg0MjhobEtvTDRnYXRXVGhNc29xMkcyRE9jUEtHbUtuUGJSQi10VmJ1ZlVrMW5w

Access Token: *AAJTSQACMDIABHR5cGUAA0pXVAACUzEAAjAx*eyJ0eXAiOiJK...


## Summary

Based on the tests above, identify which authentication method works and update the code accordingly.